<h1><center>CS 455/595a: Support Vector Machines Demos</center></h1>
Richard S. Stansbury

Reference:

[1] Aurelen Geron. *Hands on Machine Learning with Scikit-Learn & TensorFlow* O'Reilley Media Inc, 2017.

[2] Aurelen Geron. "ageron/handson-ml: A series of Jupyter notebooks that walk you through the fundamentals of Machine Learning and Deep Learning in python using Scikit-Learn and TensorFlow." Github.com, online at: https://github.com/ageron/handson-ml [last accessed 2019-03-01]

**Table of Contents**
1. [Linear Support Vector Machines](#Linear-Support-Vector-Machines)
    * [Linear SVM Classifier](#Linear-SVM-Classifier)
    * [Soft Margin Classifier](#Soft-Margin-Classifier) 
    * [Linear Kernel SVM Regression](#Linear-SVM-Regression)
    
2. [Non-Linear Support Vector Machines](#Non-Linear-Support-Vector-Machines)
    * [Polynomial Kernel SVM Classifier](#Polynomial-Kernel)
    * [Similarity Features](#Similarity-Features)
    * [Gaussian RBF Kernel SVM Classifier](#Gaussian-RBF-Kernel)
    * [Polynomial Kernel SVM Regression](#Polynomial-Kernel-SVM-Regression)

# Linear Support Vector Machines

## Linear SVM Classifier

In [13]:
# Includes
from sklearn.svm import SVC, LinearSVC
from sklearn import datasets
from matplotlib import pyplot as plt

In [21]:
# Set-up Data Set - Iris petal length and width to classify between setosa and versicolor
iris_ds = datasets.load_iris()
X = iris_ds["data"][:, (2,3)]
y = iris_ds["target"]

# setosa and versicolor are labeled as 0 and 1 respectively.  So, we wil extract the 
# corresponding rows.
setosa_or_versicolor = (y==0) | (y==1)

X = X[setosa_or_versicolor]
y = y[setosa_or_versicolor]

In [22]:
# Source: https://github.com/ageron/handson-ml/blob/master/05_support_vector_machines.ipynb

# Define Method to Plot Linear SVM
def plot_svc_decision_boundary(svm_clf, xmin, xmax):
    w = svm_clf.coef_[0]
    b = svm_clf.intercept_[0]

    # At the decision boundary, w0*x0 + w1*x1 + b = 0
    # => x1 = -w0/w1 * x0 - b/w1
    x0 = np.linspace(xmin, xmax, 200)
    decision_boundary = -w[0]/w[1] * x0 - b/w[1]

    margin = 1/w[1]
    gutter_up = decision_boundary + margin
    gutter_down = decision_boundary - margin

    svs = svm_clf.support_vectors_
    plt.scatter(svs[:, 0], svs[:, 1], s=180, facecolors='#FFAAAA')
    plt.plot(x0, decision_boundary, "k-", linewidth=2)
    plt.plot(x0, gutter_up, "k--", linewidth=2)
    plt.plot(x0, gutter_down, "k--", linewidth=2)

In [ ]:
# Linear SVM Classifier Using SVC


In [ ]:
# Linear SVM Classifier Using Linear SVM


## Soft Margin Classifier

# Non-Linear Support Vector Machines

## Polynomial Features

In [33]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

_degree = 3
_C = 10

poly_svm_pipe = Pipeline([
    ("Polynomial", PolynomialFeatures(degree=_degree)),
    ("Scale", StandardScaler()),
    ("Classifier", LinearSVC(C=_C, loss="hinge"))
    ])

moons_ds = make_moons()
X, y = make_moons(n_samples=100, noise=0.15)

poly_svm_pipe.fit(X, y)

C:\Users\richard.stansbury\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
     steps=[('Polynomial', PolynomialFeatures(degree=3, include_bias=True, interaction_only=False)), ('Scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('Classifier', LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

## Polynomia Kernel

## Similarity Features

## Gaussian RBF Kernel